In [1]:
#imports necessary packages for analysis 
#please run the following in the PMT environment with a CUDA PC
wdir='C:\\Users\\Tony\\PRT\\' # changes working directory to PMT
import os
os.chdir(wdir)
from PyRodentTracks import PRT_analysis,PRT_train
import numpy as np
import pandas as pd
import time 

**Trainning Example**

In [12]:
#create config file for trainning and analysis 
train_sample = PRT_train('D:/trainer/',creat_new=True)
#edit the config.ini file at location to desire settings and the load 
#please read the setup pdf for more details on the parameters 
train_sample.load_config()

finished creating config.ini file at {path2config}
please edit config files to desired settings and load with load_config()


In [16]:
#extract trainning frames from sample videos
train_vidpath='D:/notebook_test/' 
vids2add = [train_vidpath+i for i in os.listdir(train_vidpath)] # list of absolute path to the videos
train_sample.add_vids(vids2add)
train_sample.extractframes()

Kmeans-quantization based extracting of frames from 0.0  seconds to 559.53  seconds.
Extracting and downsampling... 8393  frames from the video.


8393it [00:05, 1446.94it/s]


Kmeans clustering ... (this might take a while)
finished extracting 30 from video:
D:/notebook_test/train_vid1.avi
Kmeans-quantization based extracting of frames from 0.0  seconds to 892.0  seconds.
Extracting and downsampling... 13380  frames from the video.


13380it [00:09, 1455.14it/s]


Kmeans clustering ... (this might take a while)
finished extracting 30 from video:
D:/notebook_test/train_vid2.avi
Kmeans-quantization based extracting of frames from 0.0  seconds to 611.0  seconds.
Extracting and downsampling... 9165  frames from the video.


9165it [00:06, 1457.65it/s]


Kmeans clustering ... (this might take a while)
finished extracting 30 from video:
D:/notebook_test/train_vid3.avi
Finished extracting frames from videos
label frames with labelimg using label_imgs() or use another too1649974680_img4l which generates Pascal VOC format


In [18]:
#opens up label_imgs gui to label the imgs
#please refere to the pdf guide for correct label settings
train_sample.label_imgs()

In [19]:
#generates a trainning folder to use to train weights in darknet or colab
#please refer to the guide pdf on using darknet (install/docker)
#the colab notebook for trainning is on our github in the notebook directory
#please look forward for future versions where trainning will be done in python
train_sample.generate_trainset()

Please transfer folder to train in darknet, in the colab notebook or docker provided


**Analysis Example of Tracker_rpi Data**

In [2]:
#Starting the analysis process for PMT
#load the config file and the video folder of interest
#for mutiple mice, we can only analyze data in the format 
#generated from the tracker_rpi
#for the configurations please follow the pdf guide
#example of video containning 4 mice
config_path = 'D:/trainer/config.ini'
test_vid = 'D:/notebook_test/test1/'
test=PRT_analysis(test_vid,config_path)

Already analyzed. Found RFID_tracks.csv file in path
Loaded RFID_tracks.csv at path D:/notebook_test/test1/


In [23]:
#performs the yolov4 detections and saves it in a csv
test.detect_mice()

Starting to yolov4 and motion detection process on  D:/notebook_test/test1//raw.avi


100%|██████████████████████████████████████████████████████████████████████████████| 9894/9894 [03:27<00:00, 47.57it/s]

Mice Detection and motion detection complete for folder D:/notebook_test/test1/
Results saved in D:/notebook_test/test1//yolo_dets.csv


In [4]:
#complete analysis pipeline
test.load_RFID() #loads the readings from rfid readers 
test.load_dets() #loads the detections made by Yolo
test.RFID_match() #starts the Sort-ID generation and matching with RFID readings
test.find_activte_mice() # finds mice that were active
test.compile_travel_trjectories() #generates individual travel trajectories of each mouse

{'RFID_readers': {5: [55, 79, 133, 165], 4: [214, 59, 330, 168], 3: [418, 65, 492, 147], 2: [193, 198, 327, 307], 1: [54, 245, 156, 344], 0: [471, 233, 512, 294]}, 'entrance_time_thres': 1.5, 'entrance_distance': 30.0, 'correct_iou': 0.06, 'RFID_dist': 70.0, 'entr_frames': 15, 'reader_thres': 0.12, 'trac_interpolation': 5, 'itc_slack': 0.1, 'entrance_reader': 0}
196 Tags were read in 600.2127611637115 seconds


Checking frames for occlusions/yolov4 fails: 100%|██████████████████████████████| 9893/9893 [00:00<00:00, 57019.84it/s]
Checking Kalmen filter predictions at frames with occlusions/yolov4 fails: 100%|████| 575/575 [00:00<00:00, 984.59it/s]


Removed sort ids [1, 5, 12, 28, 30, 31, 48, 58] for possible false positives


Starting to match RFID readings from entrance reader: 100%|████████████████████████████| 79/79 [00:03<00:00, 25.10it/s]
Starting to match RFID readings from Cage readers: 100%|█████████████████████████████| 117/117 [00:03<00:00, 31.15it/s]
Matching last tag in frame: 100%|█████████████████████████████████████████████████████| 33/33 [00:00<00:00, 390.54it/s]


87.65741923708707 % of mice detected were matched with an RFID tag


Generating Individual Rodent Trajectory:   0%|                                                   | 0/4 [00:00<?, ?it/s]

Generating Individual Rodent Trajectory:  25%|██████████▊                                | 1/4 [00:00<00:01,  1.66it/s]

Generating Individual Rodent Trajectory:  50%|█████████████████████▌                     | 2/4 [00:01<00:01,  1.47it/s]

Generating Individual Rodent Trajectory:  75%|████████████████████████████████▎          | 3/4 [00:02<00:00,  1.38it/s]

Generating Individual Rodent Trajectory: 100%|███████████████████████████████████████████| 4/4 [00:02<00:00,  1.38it/s]


In [3]:
#Generates track video for verifications
test.generate_track_video() 

**Analysis of Single Mouse Video Not Generated by Tracker_rpi**

In [ ]:
#No RFID readings required
#assumes a certain frame rate for videos
#as normal videos do not have timestamp for frames

In [2]:
import video_formater as vf
vid_path=vf.convert2pmt('D:/notebook_test/single_test.avi',30,vid_type='.avi')#vid_type can take in .mp4 or .avi

In [4]:
#performs the analysis with the normal PMT pipeline
config_path = 'D:/trainer/config.ini'
test=PRT_analysis(vid_path,config_path)
test.detect_mice() #detects mice
test.load_RFID() #loads the readings from rfid readers 
test.load_dets() #loads the detections made by Yolo
test.RFID_match() #starts the Sort-ID generation and matching with RFID readings
test.find_activte_mice() # finds mice that were active
test.compile_travel_trjectories() #generates individual travel trajectories of each mouse

Starting to yolov4 and motion detection process on  D:/notebook_test/single_test/raw.avi


100%|██████████████████████████████████████████████████████████████████████████████| 8758/8758 [03:19<00:00, 43.99it/s]

Mice Detection and motion detection complete for folder D:/notebook_test/single_test
Results saved in D:/notebook_test/single_test/yolo_dets.csv
0 Tags were read in 0.0002918243408203125 seconds



Checking frames for occlusions/yolov4 fails: 100%|█████████████████████████████| 8758/8758 [00:00<00:00, 547398.36it/s]
Checking Kalmen filter predictions at frames with occlusions/yolov4 fails: 0it [00:00, ?it/s]


Removed sort ids [] for possible false positives


Matching last tag in frame: 100%|████████████████████████████████████████████████| 7354/7354 [00:04<00:00, 1767.79it/s]


100.0 % of mice detected were matched with an RFID tag
csv file saved at D:/notebook_test/single_test/RFID_tracks.csv


Generating Individual Rodent Trajectory:   0%|                                                   | 0/1 [00:00<?, ?it/s]

Generating Individual Rodent Trajectory: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00,  1.79it/s]


In [5]:
#Generates track video for verifications
test.generate_track_video()